In [1]:
import json
import sqlite3
import pandas as pd

## Read Excel, make `coords` column for collision lookup

Workshops times (*horarios*) are modeled as a categoricall int tuple `( <day time>, <week day> )`:

- Day times
    - `0` first workshops block
    - `1` second workshops block
- Week days
    - `0` First week day "*Lunes*"
    - `1` Second week day "*Martes*"
    - ...


In [2]:
def ws_table(drop_c = False, nan_names=None):
    t = pd.read_excel('talleres.ods', sheet_name='talleres_ano')
    if nan_names is not None:
        t.loc[t['name'].isna(), 'name'] = nan_names
    t['coords'] = t.loc[:,'c1':'c3'].apply(row_coords, axis=1)
    if drop_c:
        t = t.drop(columns = t.loc[:,'c1':'c3'].columns)
    return t

def tup_from_cell_coord(s):
    assert isinstance(s,str)
    day_idx = {
        'lu': 0,
        'ma': 1,
        'mi': 2,
        'ju': 3,
        'vi': 4,
           }
    d, h = s[:2], s[2:]
    return (int(h)-1, day_idx[d.lower()])

def row_coords(row):
    r = row[row.notna()]
    return r.apply(tup_from_cell_coord).to_list()

t = ws_table(drop_c = True)
t.sample(5)

,period,name,teacher,cycle,full_name,description,coords
213,5,Inglés A,Francisca U.,canelos y manios,NaN,NaN,"[(1, 3)]"
212,5,Artes,Francisca U.,canelos y manios,NaN,NaN,"[(0, 1)]"
163,5,NaN,Javiera,canelos y manios,NaN,NaN,"[(0, 1)]"
55,2,Compost,Alejandro,canelos y manios,NaN,NaN,"[(0, 0)]"
114,4,Filosofía Kio Agus,Paula,ulmos,NaN,NaN,"[(0, 1), (0, 4)]"


## Collision of workshop coordinates

Make a boolean dataframe that test for coincidence in workshops `coords` list.

`collision_df(series)` returns the boolean coincidence table beetwen every workshop pair coordinates pair. 

In [3]:
def sample_collision(sample, df):
    return (df.astype("string") == str(sample)).any(axis=1)
    
def row_collision(row, df):
    bol = row.apply(sample_collision, df=df).any()
    return bol #df.loc[bol].index

def collision_df(series):
    df = series.apply(pd.Series)
    return df.apply(row_collision, df=df, axis=1)

t = ws_table(drop_c = True)
t = t[(t.cycle == 'ulmos') & (t.period == 1)]
collision_df(t.coords)

,0,1,6,9,10,11,15,20,21,26,31,32,33,34,39,40,47,48
0,True,False,False,False,False,True,False,True,False,False,False,False,False,False,True,False,False,False
1,False,True,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False
6,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
9,False,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False
10,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
11,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
15,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False
20,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False
21,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False
26,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [4]:
t.loc[[9,1]]

,period,name,teacher,cycle,full_name,description,coords
9,1,Jardineria y cultivo,Cristina,ulmos,Jardineria y cultivo de plantas,NaN,"[(1, 0), (1, 4)]"
1,1,Color y expresión,Carla,ulmos,NaN,NaN,"[(0, 1), (1, 4)]"


## Json serialize

In [5]:
t = ws_table(drop_c = True)
t = t[t.cycle == 'canelos y manios']
json_str = collision_df(t.coords).to_json()
print(f'{json_str[:500]} ...')

{"2":{"2":true,"3":false,"4":false,"7":false,"12":false,"13":false,"14":false,"16":false,"17":false,"22":false,"23":false,"27":false,"35":false,"36":false,"41":false,"42":false,"49":true,"50":false,"51":false,"55":false,"60":false,"61":false,"65":true,"66":false,"67":false,"72":false,"73":false,"74":false,"76":false,"77":false,"82":false,"83":false,"90":false,"91":false,"96":false,"97":false,"104":true,"105":false,"106":false,"110":true,"111":false,"112":false,"115":true,"116":false,"121":true," ...


## Label format for input switches view

In [6]:
col_names = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']

def html_description(tup, short=False):
    h_coord, d_coord = tup
    day_str = col_names[d_coord]
    if short:
        text = day_str
    else:
        time_str = f'{" a ".join(row_names[h_coord])}'
        text = f'{day_str} de {time_str}'
    return text

t = ws_table(drop_c = True)
t = t[(t.cycle == 'canelos y manios') & (t.period == 1)]
s = t.loc[7]
horarios = ' y '.join([
            html_description(tup, short=True)
            for tup in s.coords])
print(f'{s["name"]} con {s.teacher} ({horarios})')

Entre viñetas con Grace (Miércoles y Viernes)


## Sort workshops by week day and time

In [7]:
def sort_by_day_time(t):
    df = pd.DataFrame(t.coords.str[0].to_list(), index=t.index)
    idx = df.sort_values([1,0]).index
    return t.loc[idx]

t = ws_table(drop_c = True)
t = t[(t.cycle == 'canelos y manios') & (t.period == 1)]
t = sort_by_day_time(t)
t

,period,name,teacher,cycle,full_name,description,coords
12,1,Jardineria y cultivo,Cristina,canelos y manios,Jardineria y cultivo de plantas,NaN,"[(0, 0), (0, 3)]"
27,1,Explorando la naturaleza,Alejandro,canelos y manios,Historia de la pintura y filosofía,NaN,"[(0, 0)]"
35,1,Percusión corporal,Marcelo,canelos y manios,NaN,NaN,"[(0, 0)]"
2,1,Pintura I,Carla,canelos y manios,NaN,NaN,"[(1, 0)]"
49,1,Soy naturaleza,Paula,canelos y manios,NaN,NaN,"[(1, 0)]"
16,1,Fractángulos,Salvador,canelos y manios,NaN,NaN,"[(0, 1)]"
22,1,Ave y fauna,Javiera,canelos y manios,NaN,NaN,"[(0, 1)]"
41,1,Inglés,Francisca U.,canelos y manios,NaN,NaN,"[(0, 1), (1, 3)]"
13,1,Cocina,Cristina,canelos y manios,NaN,NaN,"[(1, 1)]"
50,1,Juegos de mesa,Paula,canelos y manios,NaN,NaN,"[(1, 1)]"


## Group workshops by coords for input switches view

Need to group workshops by day and time in order to present within same title in the web input switches selection view. The first coordinate of every workshop for certain cycle is used for generating an input switch. The remaining coordinates of every workshop are used to write a small text within the corresponding title but without a switch (every workshop is selected only in it first day and time).

In [8]:
t = ws_table(drop_c = True)
t = t[(t.cycle == 'ulmos')&(t.period == 1)]
t = sort_by_day_time(t)
g = t.groupby([t.coords.str[0].str[1], t.coords.str[0].str[0]]).groups
{k: [f'ws_{i}' for i in v] for k,v in g.items()}

{(0, 0): ['ws_0', 'ws_20', 'ws_39'],
 (0, 1): ['ws_9', 'ws_31'],
 (1, 0): ['ws_1', 'ws_34', 'ws_47'],
 (1, 1): ['ws_26'],
 (2, 0): ['ws_32', 'ws_40'],
 (2, 1): ['ws_10', 'ws_15'],
 (3, 0): ['ws_6', 'ws_21'],
 (3, 1): ['ws_11'],
 (4, 0): ['ws_33', 'ws_48']}

## Week schedule list for week schedule view

Use `week_schedule(d)` to make a complete week schedule dict from a partial dict `d`. Use `week_table_rows(d)` for output `d` dict values in row list ready to use in a table template.

In [9]:
days_list = ['Lu', 'Ma', 'Mi', 'Ju', 'Vi']
times_list = [('10:15', '11:15'), ('12:30', '13:30')]

workshop_times = {
    (1,3): 'Música y filosofía',
    (0,2): 'Explorando la naturaleza I',
    (1,1): 'Artes A',
    (1,4): 'Inglés',
}

def week_schedule(d={}):
    new = {(a,b): '' for a in range(len(times_list))
                         for b in range(len(days_list))}
    return {**new, **d}

def week_table_rows(d):
    return [[d[(a,b)] for b in range(len(days_list))]
                          for a in range(len(times_list))]

d = week_schedule(workshop_times)
for a,b in zip(times_list, week_table_rows(d)):
    print(a, b)

('10:15', '11:15') ['', '', 'Explorando la naturaleza I', '', '']
('12:30', '13:30') ['', 'Artes A', '', 'Música y filosofía', 'Inglés']


In [10]:
workshop_times = {(a,b): [] for a in range(len(times_list))
                         for b in range(len(days_list))}
t = ws_table(drop_c = True)
post = {10: True, 23: False, 29: True, 2: False, 38: True, 13: True, 19: True, 32: False, 11: True, 37: False, 6: False, 3: True, 28: False, 14: False, 20: False, 33: False}
idx = [i for i in post if post[i]]

for i in post:
    if post[i]:
        for coord in t.loc[i,'coords']:
            workshop_times[coord].append(t.loc[i,'name'])

{k: (v[0] if len(v)>0 else '')
 for k, v in workshop_times.items()
 }

{(0, 0): '',
 (0, 1): '',
 (0, 2): '',
 (0, 3): 'Acondicionamiento físico',
 (0, 4): 'Biodiversidad alimentaria I',
 (1, 0): '',
 (1, 1): 'Cocina',
 (1, 2): 'Reciclaje de plásticos',
 (1, 3): 'Cocina',
 (1, 4): ''}

## Sqlite DataBase

Python sqlite3 tutorial with some real data subset.

In [23]:
#This create the file inmediatly if doesn't exist
con = sqlite3.connect('wshop.db')

In [24]:
cur = con.cursor()
cur.execute("DELETE FROM taller")
con.commit()

In [25]:
cur.execute("""
    CREATE TABLE  IF NOT EXISTS
    taller (nombre, profe, ciclo)""")

In [26]:
res = cur.execute("SELECT name from sqlite_master")
res.fetchone()

('taller',)

In [27]:
cur.execute("""
    INSERT INTO taller VALUES
        ('Tejido', 'Javiera', 'ulmos'),
        ('Reciclaje de plásticos','Cristina', 'ulmos'),
        ('Artes', 'Francisca U.', 'ulmos')
""")

In [28]:
con.commit()

In [29]:
res = cur.execute("SELECT nombre FROM taller")
res.fetchall()

[('Tejido',), ('Reciclaje de plásticos',), ('Artes',)]

In [30]:
t = ws_table(drop_c = True)
data = []
for i in range(10,15):
    row = tuple([
        t.loc[i,col] for col in ['name','teacher','cycle']])
    data.append(row)

cur.executemany("INSERT INTO taller VALUES(?,?,?)",
                data)
con.commit()
#t.loc[10:20,['name','teacher','cycle']].to_list()
# s = t.loc[40]
# print(f'{s["name"]} con {s.teacher}, horarios: {s.coords}')
# s

In [31]:
for row in cur.execute(
    "SELECT nombre, profe, ciclo FROM taller ORDER BY ciclo"):
    print(row)

('Jardineria y cultivo', 'Cristina', 'canelos y manios')
('Cocina', 'Cristina', 'canelos y manios')
('Reciclaje de plásticos', 'Cristina', 'canelos y manios')
('Tejido', 'Javiera', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Artes', 'Francisca U.', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Cocina', 'Cristina', 'ulmos')


In [32]:
con.close()
new_con = sqlite3.connect("wshop.db")
new_cur = new_con.cursor()
res = new_cur.execute(
    "SELECT nombre, profe, ciclo FROM taller")
for row in res:
    print(row)

('Tejido', 'Javiera', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Artes', 'Francisca U.', 'ulmos')
('Reciclaje de plásticos', 'Cristina', 'ulmos')
('Cocina', 'Cristina', 'ulmos')
('Jardineria y cultivo', 'Cristina', 'canelos y manios')
('Cocina', 'Cristina', 'canelos y manios')
('Reciclaje de plásticos', 'Cristina', 'canelos y manios')


In [33]:
new_cur.execute("DELETE FROM taller")
new_con.commit()
new_con.close()

### Using wshop.db inscription table

In [34]:
con = sqlite3.connect('wshop.db')
cur = con.cursor()
t = ws_table(nan_names='(No definido)')
ids = [f"ws_{i}" for i in t.index]
ids_str = ', '.join(ids)
cur.execute(f"""
    CREATE TABLE IF NOT EXISTS
    inscription (sname, cycle, {ids_str})
    """)
con.commit()
con.close()

In [35]:
def insert_to_dbase(sname, cycle, idx, t):
    idx_bool_list = [0 for _ in t.index]
    for i in idx:
        idx_bool_list[i] = 1
    idx_str = ', '.join([str(i) for i in idx_bool_list])
    with con:
        con.execute(f"""
            INSERT INTO inscription
            VALUES ('{sname}', '{cycle}', {idx_str})
            """)

con = sqlite3.connect('wshop.db')
t = ws_table(nan_names='(No definido)')
insert_to_dbase('DiegoS', 'coihues y avellanos',
                [113,198,140,197,226,155,249], t)
con.close()

In [36]:
def read_from_dbase(con, t):
    with con:
        res = con.execute("SELECT * FROM inscription")
    by_unique_name = {}
    for row in res.fetchall():
        by_unique_name[row[0]] = row[2:]
    ins = []
    for boolsidx in by_unique_name.values():
        ins.extend([i for i,v in enumerate(boolsidx) if v == 1])
    return [ins.count(i) for i in t.index]

con = sqlite3.connect('wshop.db')
t = ws_table(drop_c=True, nan_names='(No definido)')
t.loc[:,'applied'] = read_from_dbase(con, t)
con.close()
t[['name','teacher', 'cycle', 'applied']].head(10)

,name,teacher,cycle,applied
0,Pintura,Carla,ulmos,0
1,Color y expresión,Carla,ulmos,0
2,Pintura I,Carla,canelos y manios,0
3,Color y expresión I,Carla,canelos y manios,1
4,Títeres,Carla,canelos y manios,1
5,Historia del arte,Carla,coihues y avellanos,3
6,Exploradores(as) de derechos,Grace,ulmos,0
7,Entre viñetas,Grace,canelos y manios,0
8,Resolución de conflictos,Grace,coihues y avellanos,2
9,Jardineria y cultivo,Cristina,ulmos,1


In [ ]:
def reset_dbase(con):
    with con:
        con.execute("DELETE FROM inscription")

con = sqlite3.connect('wshop.db')
reset_dbase(con)
con.close()